In [1]:
!pip install z3-solver

In [5]:
from z3 import *


def always_holds(condition):
    s = Solver()
    s.add(Not(condition))
    return s.check() == unsat


# Verify invariance
def verify(predicates, init_state, transition):
    refuted = []
    for k in range(len(predicates)):
        predicate = predicates[k]
        
        # Check that the invariant holds at the beginning of the loop        
        s = Solver()
        s.add(init_state, predicate)
        if s.check() == unsat:
            refuted.append(predicate)
            continue



        # Check the predicate hold inductively 
        context = True
        for j in range(len(predicates)):
            context = And(context, predicates[j])
        inductive_predicate = substitute(predicate, [(x, x_next), (y, y_next), (z, z_next), (i, i_next)])
        inductive_check = ForAll([x, y, z, i], Implies(And(context, transition), inductive_predicate))
        # print(inductive_check, always_holds(inductive_check))
        if not always_holds(inductive_check):
            refuted.append(predicate)
    return refuted



def houdini_algorithm(predicates, init_state, transition):
    current_predicates = predicates.copy()
    while True:
        print("start_predicates:", current_predicates)
        refuted = verify(current_predicates, init_state, transition)
        if len(refuted) == 0:
            break
        current_predicates = [p for p in current_predicates if p not in refuted]
        print("end_predicates:", current_predicates)
    return current_predicates


x, y, z, tmp, i = Ints('x y z tmp i')
x_next, y_next, z_next, i_next = Ints('x_next y_next z_next i_next')

# init_state 
init_state = And(x == 1, y == 2, z == 3, i == 0)

# body of loop
transition = And(i < 10, tmp == x, x_next == y, y_next == z, z_next == tmp, i_next == i + 1)


predicates = [i == 0, i < 10, i <= 10, i > 10, x != y, And((z != y), (x != z))]


final_invariant = houdini_algorithm(predicates, init_state, transition)
print("Final invariant:", final_invariant)


start_predicates: [i == 0, i < 10, i <= 10, i > 10, x != y, And(z != y, x != z)]
end_predicates: [i == 0, i < 10, i <= 10, x != y, And(z != y, x != z)]
start_predicates: [i == 0, i < 10, i <= 10, x != y, And(z != y, x != z)]
end_predicates: [i < 10, i <= 10, x != y, And(z != y, x != z)]
start_predicates: [i < 10, i <= 10, x != y, And(z != y, x != z)]
end_predicates: [i <= 10, x != y, And(z != y, x != z)]
start_predicates: [i <= 10, x != y, And(z != y, x != z)]
Final invariant: [i <= 10, x != y, And(z != y, x != z)]
